<a href="https://colab.research.google.com/github/BraedynL0530/PortfolioWebsite/blob/master/Multi_head_classfication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from torch.autograd import forward_ad
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
from dataclasses import  dataclass
torch.manual_seed(42)

#config
@dataclass
class config:
  vocab_size: int  #unique words
  block_size: int  #how far back(context) it can see, memory/ how many tokens back
  n_layer: int   # stacked blocks, more layers more reasoning more train time
  n_head: int   # attentions per layer, how many "eyes" looking for a new pattern
  n_embd: int   #size of vector for each token
  dropout: float  #prevents overfitting by stopping random paths
  pad_token_id: int



"""
self attention: part 1 of transformer
Q K V, query key value. helps use the two embeddings to learn diffrent meanings for words and give the diffrent vectors even if the same word
below is theory class is optimized, it condences the prjections into one huge vector and splits. other than that its nearly identical just more efficent
"""

"""
#learnable compenets
q_prog = nn.Linear(C, C, bias =False)
k_prog = nn.Linear(C, C, bias =False)
v_prog = nn.Linear(C, C, bias =False)

#weights
q_prog.weight.data = torch.randn(C,C)
q_prog.weight.data = torch.randn(C,C)
q_prog.weight.data = torch.randn(C,C)

#preform projection
q = q_prog(x)
k = k_prog(x)
v = v_prog(x)

scores = q @ k.transpose(-2,-1)
print("scores",scores)



Attention(Q,K,V)=softmax(​QK^⊤/dk​​)V

d_k = k.size(-1)#last dimesion of
scaled_scores = scores / math.sqrt(d_k)
attention_weights = F.softmax(scaled_scores, dim=1)
print("scaled scores", scaled_scores)
print("scaled scores -> percentages", attention_weights)

# aggreation Last part of attention!
output = attention_weights @ v
print("output!:",output)

"""

# Core logic for MultiHead
class CausalSelfAttention(nn.Module): #no longer casual masking, bi directional
  def __init__(self, config :config):
    super().__init__()
    assert config.n_embd % config.n_head == 0
    self.n_head = config.n_head
    self.n_embd = config.n_embd
    self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=False) # Fuzed layer = more efficent
    self.attn_drop = nn.Dropout(config.dropout)
    self.register_buffer( # part of causal masking
        "bias",# buffer name
        torch.tril(torch.ones(config.block_size,config.block_size))
        .view(1,1, config.block_size, config.block_size)
    )

    self.c_proj = nn.Linear(config.n_embd,config.n_embd)

  def forward(self, x,pad_mask=None):
    B, T, C = x.size()
    head_dim = C // self.n_head

    # project once -> split
    qkv = self.c_attn(x)
    q, k, v = qkv.split(C, dim=2)

    # reshape into heads
    q = q.view(B, T, self.n_head, head_dim).transpose(1, 2)
    k = k.view(B, T, self.n_head, head_dim).transpose(1, 2)
    v = v.view(B, T, self.n_head, head_dim).transpose(1, 2)

    # attention
    att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(head_dim))
    #att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float("-inf")) # prevents it from seeing future tokens. Removed for bidirectional

    # Prevent attending to padding tokens (BEFORE softmax now)
    if pad_mask is not None:
      att = att.masked_fill(
          pad_mask[:, None, None, :T] == 0,
          float("-inf")
      )

    att = F.softmax(att, dim=-1)
    att = self.attn_drop(att)

    # aggregate :3
    y = att @ v

    # merge heads
    y = y.transpose(1, 2).contiguous().view(B, T, C)

    # final projection
    y = self.c_proj(y)
    return y


class MLP(nn.Module):
  def __init__(self, config :config):
    super().__init__()
    self.fc = nn.Linear(config.n_embd, 4 * config.n_embd) #expands dimestions, think of it as more room to think / combining features
    self.proj = nn.Linear(4 * config.n_embd, config.n_embd) # condenses back so it can be added back to attetion
    self.drop = nn.Dropout(config.dropout) #refer to config

  def forward(self, x):
    x = self.fc(x)
    x = F.gelu(x) # makes x nonlinear so fc and proj dont just merge into one straight line
    x =self.proj(x)
    x = self.drop(x)

    return x

class Block(nn.Module): #residual connection
  def __init__(self, config : config): #litterly just does f(x) + x instead of f(x) so mlp dosesnt relearn it takes the learned/trained data and keeps it
    super().__init__()
    self.ln_1 = nn.LayerNorm(config.n_embd)
    self.attn = CausalSelfAttention(config)
    self.ln_2 = nn.LayerNorm(config.n_embd)
    self.mlp = MLP(config)

  def forward(self, x, pad_mask=None):  # ← Added pad_mask parameter
    # focus (the "+")
    x = x + self.attn(self.ln_1(x), pad_mask=pad_mask)
    x = x + self.mlp(self.ln_2(x))
    return x

In [3]:

class NLP(nn.Module):
  def __init__(self, config: config):
    super().__init__()
    # Input
    self.wte = nn.Embedding(config.vocab_size, config.n_embd)
    self.wpe = nn.Embedding(config.block_size, config.n_embd)
    self.drop = nn.Dropout(config.dropout)
    self.config = config
    self.pad_token_id = config.pad_token_id




    # Processing, makes a stack/block / LAYER for deeper understanding
    # Data flows through sequncesnsy so more refined/better understanding
    self.h = nn.ModuleList([Block(config) for _ in range(config.n_layer)])

    #output layers
    self.ln_f = nn.LayerNorm(config.n_embd) # final layer norm
    d_model=config.n_embd
    num_genres = 0#temp till i decied
    num_tools = 0 #also temp
    num_purposes = 0 #yk why

    self.token_head = nn.Linear(d_model,num_tools) #seq label

    self.pool_vector = nn.Linear(d_model,1) #attention pooling

    # classification heads
    self.genre_head = nn.Linear(config.n_embd, num_genres)
    self.purpose_head = nn.Linear(config.n_embd, num_purposes)

    self.apply(self._init_weights)


  def _init_weights(self, module):
    if isinstance(module, nn.Linear):
        nn.init.normal_(module.weight, mean=0.0, std=0.02)
        if module.bias is not None:
            nn.init.zeros_(module.bias)
    elif isinstance(module, nn.Embedding):
        nn.init.normal_(module.weight, mean=0.0, std=0.02)


  def attention_pool(self, x, pad_mask):
        """
        Weighted pooling using attention
        x: [batch, seq_len, embd]
        pad_mask: [batch, seq_len]
        """
        # Compute attention scores for each token
        attn_scores = self.pool_attention(x).squeeze(-1)  # [batch, seq_len]

        # Mask out padding tokens
        attn_scores = attn_scores.masked_fill(pad_mask == 0, float('-inf'))

        # Softmax to get weights
        attn_weights = F.softmax(attn_scores, dim=1).unsqueeze(-1)  # [batch, seq_len, 1]

        # Weighted sum
        pooled = (x * attn_weights).sum(dim=1)  # [batch, embd]

        return pooled

  def forward(self, idx): #idx = input targets = inputs shifter one pos to left
    B, T = idx.size()


    tok_emb = self.wte(idx)
    tok_emb[idx == self.pad_token_id] = 0
    pos = torch.arange(T, device=idx.device).unsqueeze(0)
    pos_emb = self.wpe(pos)
    x = self.drop(tok_emb + pos_emb)

    pad_mask = (idx != self.pad_token_id).float()  # ← Mask: 1 for real tokens, 0 for padding

    # Process through transformer blocks
    for block in self.h:
        x = block(x, pad_mask=pad_mask)  # ← Pass mask through each block

    # Final layer norm
    x = self.ln_f(x)

    token_logits = self.token_head(x)

    doc_rep = self.attention_pool(x,pad_mask)
    genre_logits = self.genre_head(doc_rep)
    purpose_logits = self.purpose_head(doc_rep)

    return token_logits, genre_logits, purpose_logits



In [ ]:
"""
Data Cleaning Utilities
========================

Cleans messy README text before feeding to model.
Handles: Unicode, emojis, special chars, formatting artifacts
"""

import re
import unicodedata

def clean_readme(text):
    """
    Clean README text for better model training.

    Handles:
    - Remove excessive whitespace/newlines
    - Strip emojis (they don't help classification)
    - Normalize unicode
    - Remove code blocks (they're noise)
    - Keep only ASCII + common punctuation
    """

    # Remove code blocks (they're just noise for classification)
    text = re.sub(r'```[\s\S]*?```', '', text)
    text = re.sub(r'`[^`]+`', '', text)

    # Remove markdown links but keep text: [text](url) -> text
    text = re.sub(r'\[([^\]]+)\]\([^\)]+\)', r'\1', text)

    # Remove image markdown: ![alt](url)
    text = re.sub(r'!\[[^\]]*\]\([^\)]+\)', '', text)

    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)

    # Remove URLs
    text = re.sub(r'http[s]?://\S+', '', text)

    # Normalize unicode (convert fancy chars to ASCII equivalents)
    text = unicodedata.normalize('NFKD', text)

    # Remove emojis and other unicode symbols
    # Keep only: letters, numbers, basic punctuation, spaces
    text = re.sub(r'[^\w\s\.\,\!\?\:\;\-\(\)\[\]\/\#\@\+\=\&]', ' ', text)

    # Remove excessive whitespace
    text = re.sub(r'\s+', ' ', text)

    # Remove tabs
    text = text.replace('\t', ' ')

    # Strip leading/trailing whitespace
    text = text.strip()

    return text

def extract_tools_from_text(text):
    """
    Try to extract tool names from README using patterns.
    This is weak supervision - not perfect but helpful.

    Common patterns:
    - "Built with X, Y, Z"
    - "Technologies: X, Y, Z"
    - "Uses: X"
    """

    tools = set()
    text_lower = text.lower()

    # Pattern 1: "Built with/using X"
    built_with = re.findall(r'built (?:with|using)[\s:]+([^\n\.\!]{5,100})', text_lower)
    for match in built_with:
        # Split on commas, "and", slashes
        parts = re.split(r'[,/]|\sand\s', match)
        tools.update([p.strip() for p in parts if p.strip()])

    # Pattern 2: "Technologies: X, Y, Z"
    tech_section = re.findall(r'technolog(?:ies|y)[\s:]+([^\n\.\!]{5,100})', text_lower)
    for match in tech_section:
        parts = re.split(r'[,/]|\sand\s', match)
        tools.update([p.strip() for p in parts if p.strip()])

    # Pattern 3: "Uses: X"
    uses_section = re.findall(r'uses[\s:]+([^\n\.\!]{5,100})', text_lower)
    for match in uses_section:
        parts = re.split(r'[,/]|\sand\s', match)
        tools.update([p.strip() for p in parts if p.strip()])

    return list(tools)

def validate_labels(tools, genres, purposes, tool_vocab, genre_vocab, purpose_vocab):
    """
    Check if labels are valid (exist in vocabulary).
    Warns about typos/unknown labels.
    """

    invalid = []

    for tool in tools:
        if tool not in tool_vocab:
            invalid.append(f"Unknown tool: '{tool}'")

    for genre in genres:
        if genre not in genre_vocab:
            invalid.append(f"Unknown genre: '{genre}'")

    for purpose in purposes:
        if purpose not in purpose_vocab:
            invalid.append(f"Unknown purpose: '{purpose}'")

    return invalid

# ============================================================================
# USAGE IN TRAINING PIPELINE
# ============================================================================

def preprocess_training_data(data_path, output_path):
    """
    Clean all READMEs in training data.

    Before:
    {
        "readme": "Apns4j\n\nLicense\n\nBuild Status\n\n..."
    }

    After:
    {
        "readme": "Apns4j License Build Status Quick start..."
    }
    """
    import json

    with open(data_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    cleaned_count = 0

    for item in data:
        if 'readme' in item:
            original = item['readme']
            cleaned = clean_readme(original)

            if len(cleaned) < 50:  # Too short after cleaning
                print(f"⚠️  '{item.get('name', 'Unknown')}' README too short after cleaning")

            item['readme'] = cleaned
            cleaned_count += 1

    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

    print(f"✅ Cleaned {cleaned_count} READMEs")
    print(f"💾 Saved to {output_path}")

# ============================================================================
# GOOGLE COLAB SETUP
# ============================================================================

def setup_colab_training():
    """
    Complete setup for training in Google Colab.

    Run this first in Colab to mount Drive and install packages.
    """

    print("📦 Setting up Colab environment...")

    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Install packages
    print("Installing packages...")
    import subprocess
    subprocess.run(['pip', 'install', '-q', 'transformers', 'torch'])

    # Create directories
    import os
    base_dir = '/content/drive/MyDrive/portfolio_classifier'
    os.makedirs(base_dir, exist_ok=True)
    os.makedirs(f'{base_dir}/data', exist_ok=True)
    os.makedirs(f'{base_dir}/models', exist_ok=True)

    print(f"✅ Setup complete!")
    print(f"📁 Working directory: {base_dir}")
    print()
    print("Next steps:")
    print("1. Upload labeled_data.json to /content/drive/MyDrive/portfolio_classifier/data/")
    print("2. Upload your NLP model code")
    print("3. Run training script")

    return base_dir

# ============================================================================
# COLAB TRAINING SCRIPT
# ============================================================================

COLAB_TRAINING_SCRIPT = """
# Google Colab Training Script
# =============================
# Run this in a Colab notebook cell

# 1. Setup
from data_cleaning import setup_colab_training, preprocess_training_data, clean_readme
base_dir = setup_colab_training()

# 2. Clean your data
preprocess_training_data(
    f'{base_dir}/data/labeled_data.json',
    f'{base_dir}/data/labeled_data_clean.json'
)

# 3. Load your model (upload NLP.py to Colab first)
from NLP import NLP, config
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# 4. Initialize model
model_config = config(
    vocab_size=50257,
    block_size=512,
    n_layer=4,
    n_head=8,
    n_embd=256,
    dropout=0.1,
    pad_token_id=50256
)

model = NLP(
    model_config,
    num_tools=30,  # Adjust to your vocab sizes
    num_genres=10,
    num_purposes=15
).to(device)

# 5. Train
from train_portfolio_classifier import RepoDataset, train_epoch
from transformers import GPT2TokenizerFast
from torch.utils.data import DataLoader

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

dataset = RepoDataset(
    f'{base_dir}/data/labeled_data_clean.json',
    tokenizer,
    max_len=512,
    augment=True
)

loader = DataLoader(dataset, batch_size=4, shuffle=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
criterion = torch.nn.BCEWithLogitsLoss()

print("🚀 Starting training...")

for epoch in range(10):
    loss = train_epoch(model, loader, optimizer, criterion, device)
    print(f"Epoch {epoch+1}/10: Loss = {loss:.4f}")

    # Save checkpoint every 3 epochs
    if (epoch + 1) % 3 == 0:
        torch.save(
            model.state_dict(),
            f'{base_dir}/models/checkpoint_epoch_{epoch+1}.pt'
        )
        print(f"💾 Saved checkpoint")

# 6. Save final model
torch.save(model.state_dict(), f'{base_dir}/models/portfolio_classifier_final.pt')
print("✅ Training complete!")
print(f"📁 Model saved to: {base_dir}/models/portfolio_classifier_final.pt")
"""

if __name__ == "__main__":
    print("Data Cleaning Utilities")
    print("=" * 50)
    print()
    print("Functions available:")
    print("  clean_readme(text) - Clean messy README text")
    print("  extract_tools_from_text(text) - Auto-detect tools")
    print("  validate_labels(...) - Check label validity")
    print("  preprocess_training_data(...) - Clean entire dataset")
    print("  setup_colab_training() - Setup Google Colab environment")
    print()
    print("Example usage:")
    print()
    print("# Clean single README")
    print("clean_text = clean_readme(messy_readme)")
    print()
    print("# Clean entire dataset")
    print("preprocess_training_data('raw_data.json', 'clean_data.json')")
    print()
    print("# For Google Colab:")
    print("base_dir = setup_colab_training()")

In [ ]:
# Portfolio Classifier - Google Colab Training Notebook
# ======================================================
# Upload this to Google Colab and run cell by cell

# ============================================================================
# CELL 1: Setup Environment
# ============================================================================

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install packages
!pip install -q transformers torch

# Create directories
import os
base_dir = '/content/drive/MyDrive/portfolio_classifier'
os.makedirs(base_dir, exist_ok=True)
os.makedirs(f'{base_dir}/data', exist_ok=True)
os.makedirs(f'{base_dir}/models', exist_ok=True)
os.makedirs(f'{base_dir}/code', exist_ok=True)

print(f"✅ Setup complete!")
print(f"📁 Base directory: {base_dir}")
print()
print("Next: Upload your files to Google Drive:")
print(f"  - labeled_data.json → {base_dir}/data/")
print(f"  - NLP.py (your model) → {base_dir}/code/")
print(f"  - train_portfolio_classifier.py → {base_dir}/code/")
print(f"  - data_cleaning.py → {base_dir}/code/")

# ============================================================================
# CELL 2: Add code to path and import
# ============================================================================

import sys
sys.path.append(f'{base_dir}/code')

# Import your modules
from data_cleaning import clean_readme, preprocess_training_data
# from NLP import NLP, config  # Uncomment after uploading NLP.py

print("✅ Modules imported")

# ============================================================================
# CELL 3: Clean Data
# ============================================================================

# Option A: Clean existing data file
preprocess_training_data(
    f'{base_dir}/data/labeled_data.json',
    f'{base_dir}/data/labeled_data_clean.json'
)

# Option B: Dataset will auto-clean on load (set clean=True)
# Just use labeled_data.json directly

# ============================================================================
# CELL 4: Check GPU
# ============================================================================

import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🖥️  Using device: {device}")

if device == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  No GPU detected! Training will be slow.")
    print("   Go to: Runtime → Change runtime type → GPU")

# ============================================================================
# CELL 5: Define Model Config (copy your config here)
# ============================================================================

from dataclasses import dataclass

@dataclass
class ModelConfig:
    vocab_size: int = 50257  # GPT-2 vocab size
    block_size: int = 512
    n_layer: int = 4         # Small model for fast training
    n_head: int = 8
    n_embd: int = 256
    dropout: float = 0.1
    pad_token_id: int = 50256

# Classification vocab sizes
NUM_TOOLS = 30
NUM_GENRES = 10
NUM_PURPOSES = 15

# ============================================================================
# CELL 6: Load Model
# ============================================================================

# Import your NLP model (make sure NLP.py is in {base_dir}/code/)
from NLP import NLP

config = ModelConfig()

model = NLP(
    config,
    num_tools=NUM_TOOLS,
    num_genres=NUM_GENRES,
    num_purposes=NUM_PURPOSES
).to(device)

print(f"✅ Model loaded")
print(f"   Parameters: {sum(p.numel() for p in model.parameters()):,}")

# ============================================================================
# CELL 7: Prepare Data
# ============================================================================

from train_portfolio_classifier import RepoDataset
from transformers import GPT2TokenizerFast
from torch.utils.data import DataLoader

# Initialize tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Load dataset (with auto-cleaning)
dataset = RepoDataset(
    f'{base_dir}/data/labeled_data.json',  # or labeled_data_clean.json
    tokenizer,
    max_len=512,
    augment=True,  # Auto-detect additional tools
    clean=True     # Auto-clean messy text
)

# Create dataloader
loader = DataLoader(
    dataset,
    batch_size=4,  # Adjust based on GPU memory
    shuffle=True,
    pin_memory=True if device == 'cuda' else False
)

print(f"✅ Data loaded: {len(dataset)} examples")

# ============================================================================
# CELL 8: Training Setup
# ============================================================================

from train_portfolio_classifier import FocalLoss, train_epoch

# Optimizer
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=3e-4,
    weight_decay=0.01
)

# Loss function (handles class imbalance)
criterion = FocalLoss(alpha=0.25, gamma=2.0)

# Training config
EPOCHS = 10
SAVE_EVERY = 3  # Save checkpoint every N epochs

print("🚀 Ready to train!")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch size: {loader.batch_size}")
print(f"   Steps per epoch: {len(loader)}")

# ============================================================================
# CELL 9: Train Model
# ============================================================================

import time
from tqdm import tqdm

print("=" * 60)
print("TRAINING START")
print("=" * 60)

training_losses = []

for epoch in range(EPOCHS):
    epoch_start = time.time()

    # Train one epoch
    loss = train_epoch(model, loader, optimizer, criterion, device)
    training_losses.append(loss)

    epoch_time = time.time() - epoch_start

    print(f"\n📊 Epoch {epoch+1}/{EPOCHS}")
    print(f"   Loss: {loss:.4f}")
    print(f"   Time: {epoch_time:.1f}s")

    # Save checkpoint
    if (epoch + 1) % SAVE_EVERY == 0:
        checkpoint_path = f'{base_dir}/models/checkpoint_epoch_{epoch+1}.pt'
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
        }, checkpoint_path)
        print(f"   💾 Checkpoint saved")

print("\n" + "=" * 60)
print("TRAINING COMPLETE!")
print("=" * 60)

# ============================================================================
# CELL 10: Save Final Model
# ============================================================================

# Save model
final_model_path = f'{base_dir}/models/portfolio_classifier_final.pt'
torch.save(model.state_dict(), final_model_path)

# Save tokenizer
tokenizer_path = f'{base_dir}/models/tokenizer'
tokenizer.save_pretrained(tokenizer_path)

print(f"✅ Model saved to: {final_model_path}")
print(f"✅ Tokenizer saved to: {tokenizer_path}")
print()
print("📊 Training Summary:")
print(f"   Initial loss: {training_losses[0]:.4f}")
print(f"   Final loss: {training_losses[-1]:.4f}")
print(f"   Improvement: {training_losses[0] - training_losses[-1]:.4f}")

# ============================================================================
# CELL 11: Test Inference
# ============================================================================

from train_portfolio_classifier import predict

# Test on a README
test_readme = """
My Awesome Project

A web application built with Django and React for real-time data visualization.

Tech Stack:
- Django REST Framework
- React with TypeScript
- PostgreSQL
- Redis for caching
- Celery for background tasks

Features:
- Real-time dashboards
- User authentication
- API for external integrations
"""

print("🧪 Testing inference...")
print("=" * 60)

predictions = predict(model, test_readme, tokenizer, threshold=0.5)

print("📦 Test README:")
print(test_readme[:200] + "...")
print()
print("🤖 Predictions:")
print()
print("Genres:")
for p in predictions['genres']:
    print(f"  • {p['label']} ({p['confidence']:.1%})")

print("\nPurposes:")
for p in predictions['purposes']:
    print(f"  • {p['label']} ({p['confidence']:.1%})")

print("\n✅ Inference works!")

# ============================================================================
# CELL 12: Download Model (Optional)
# ============================================================================

# If you want to download model to local machine
from google.colab import files

# Download model
# files.download(final_model_path)

# Or just use it directly from Google Drive in your app!

print("🎉 All done!")
print()
print("Next steps:")
print("1. Test on more examples")
print("2. Use label_review_tool.py to add more training data")
print("3. Retrain with expanded dataset")
print("4. Deploy in your portfolio")

In [ ]:
"""
Portfolio Project Classifier - Full Training Pipeline
======================================================

This script trains a multi-label classifier to extract:
- Tools/technologies used (django, react, pytorch, etc.)
- Project genre (web-app, dev-tools, ml, etc.)
- Project purpose (analytics, automation, etc.)

The model uses a transformer encoder (your NLP class) with separate
classification heads for each task.
"""

import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2TokenizerFast
from tqdm import tqdm
import numpy as np
from pathlib import Path

# ============================================================================
# CONFIGURATION
# ============================================================================

class Config:
    # Vocabularies - START SMALL, expand as needed
    TOOL_VOCAB = [
        # Frontend
        "react", "vue", "angular", "svelte", "nextjs", "html", "css", "javascript", "typescript",
        # Backend
        "django", "flask", "fastapi", "express", "nodejs", "go", "rust",
        # Database
        "postgres", "mysql", "mongodb", "redis", "sqlite",
        # ML/Data
        "pytorch", "tensorflow", "sklearn", "pandas", "numpy",
        # Desktop/Mobile
        "pyqt5", "electron", "flutter", "swift",
        # DevOps
        "docker", "kubernetes", "aws", "gcp",
        # Other
        "python", "celery"
    ]

    GENRE_VOCAB = [
        "web-app", "api", "cli", "library", "dev-tools",
        "ml", "desktop", "mobile", "bot", "game"
    ]

    PURPOSE_VOCAB = [
        "analytics", "monitoring", "automation", "productivity",
        "visualization", "e-commerce", "social", "education",
        "security", "real-time", "scraping"
    ]

    # Tool synonyms for auto-labeling
    TOOL_SYNONYMS = {
        "go": ["golang", "go language"],
        "javascript": ["js", "node"],
        "python": ["py"],
        "typescript": ["ts"],
        "postgres": ["postgresql", "psql"],
        "react": ["reactjs"],
        "vue": ["vuejs"],
        "pyqt5": ["pyqt", "qt5", "qt"],
        "sklearn": ["scikit-learn", "scikit"],
        "pytorch": ["torch"],
        "nodejs": ["node.js", "node"],
    }

    # Model config
    VOCAB_SIZE = 50257  # GPT-2 tokenizer size
    BLOCK_SIZE = 512
    N_LAYER = 4  # Start small, increase if underfitting
    N_HEAD = 8
    N_EMBD = 256
    DROPOUT = 0.1

    # Training config
    BATCH_SIZE = 4
    LEARNING_RATE = 3e-4
    EPOCHS = 10
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

config = Config()

# Create label mappings
tool2idx = {tool: i for i, tool in enumerate(config.TOOL_VOCAB)}
genre2idx = {genre: i for i, genre in enumerate(config.GENRE_VOCAB)}
purpose2idx = {purpose: i for i, purpose in enumerate(config.PURPOSE_VOCAB)}

idx2tool = {i: tool for tool, i in tool2idx.items()}
idx2genre = {i: genre for genre, i in genre2idx.items()}
idx2purpose = {i: purpose for purpose, i in purpose2idx.items()}

# ============================================================================
# LABEL AUGMENTATION
# ============================================================================

def augment_tool_labels(readme_text, manual_labels):
    """
    Automatically detect additional tools from README text using string matching.
    This is "weak supervision" - not perfect but helps label faster.

    Args:
        readme_text: The full README content
        manual_labels: List of tools you manually labeled

    Returns:
        Combined list of manual + auto-detected tools
    """
    text_lower = readme_text.lower()
    auto_labels = set(manual_labels)  # Start with manual labels

    for tool, synonyms in config.TOOL_SYNONYMS.items():
        # Check if tool name or any synonym appears in text
        all_names = [tool] + synonyms
        if any(name in text_lower for name in all_names):
            auto_labels.add(tool)

    return list(auto_labels)

def labels_to_vector(label_list, vocab_dict, vocab_size):
    """
    Convert list of string labels to binary vector.

    Example:
        labels = ["django", "react"]
        vocab = {"django": 0, "flask": 1, "react": 2, ...}
        result = [1, 0, 1, 0, 0, ...] (50-dim vector)
    """
    vector = torch.zeros(vocab_size)
    for label in label_list:
        if label in vocab_dict:
            vector[vocab_dict[label]] = 1
    return vector

# ============================================================================
# DATASET
# ============================================================================

class RepoDataset(Dataset):
    """
    Dataset for training the multi-label classifier.

    Expected data format (JSON):
    [
        {
            "name": "Desktop Pet ML",
            "readme": "DESKTOP PET w/ MACHINE LEARNING...",
            "tools": ["pyqt5", "python", "sqlite"],
            "genres": ["desktop", "dev-tools"],
            "purposes": ["productivity", "automation"]
        },
        ...
    ]
    """

    def __init__(self, data_path, tokenizer, max_len=512, augment=True, clean=True):
        with open(data_path, 'r', encoding='utf-8') as f:
            self.data = json.load(f)

        self.tokenizer = tokenizer
        self.max_len = max_len
        self.augment = augment

        # Clean READMEs on load
        if clean:
            print("🧹 Cleaning READMEs...")
            from data_cleaning import clean_readme
            for item in self.data:
                if 'readme' in item:
                    item['readme'] = clean_readme(item['readme'])

        print(f"📊 Loaded {len(self.data)} examples")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        # Tokenize README (truncate if too long)
        readme = item['readme']
        tokens = self.tokenizer.encode(readme)[:self.max_len]

        # Pad to max_len
        if len(tokens) < self.max_len:
            tokens += [self.tokenizer.pad_token_id] * (self.max_len - len(tokens))

        tokens = torch.tensor(tokens, dtype=torch.long)

        # Get labels (with optional augmentation)
        tool_labels = item.get('tools', [])
        if self.augment:
            tool_labels = augment_tool_labels(readme, tool_labels)

        genre_labels = item.get('genres', [])
        purpose_labels = item.get('purposes', [])

        # Convert to binary vectors
        tools_vec = labels_to_vector(tool_labels, tool2idx, len(config.TOOL_VOCAB))
        genres_vec = labels_to_vector(genre_labels, genre2idx, len(config.GENRE_VOCAB))
        purposes_vec = labels_to_vector(purpose_labels, purpose2idx, len(config.PURPOSE_VOCAB))

        return tokens, tools_vec, genres_vec, purposes_vec

# ============================================================================
# LOSS FUNCTIONS
# ============================================================================

class FocalLoss(nn.Module):
    """
    Focal Loss - handles class imbalance better than BCE.

    Focuses training on hard examples. If a label is rare (like "game"),
    the model pays more attention to learning it.

    alpha: weight for positive examples (0.25 = focus more on negatives)
    gamma: focusing parameter (2 = strongly focus on hard examples)
    """
    def __init__(self, alpha=0.25, gamma=2):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        # BCE loss
        bce = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')

        # Probability of correct prediction
        pt = torch.exp(-bce)

        # Focal weight: (1 - pt)^gamma
        # Easy examples (pt close to 1) get low weight
        # Hard examples (pt close to 0) get high weight
        focal_weight = (1 - pt) ** self.gamma

        # Final loss
        focal_loss = self.alpha * focal_weight * bce

        return focal_loss.mean()

# ============================================================================
# TRAINING
# ============================================================================

def train_epoch(model, dataloader, optimizer, criterion, device):
    """
    Train for one epoch.

    Returns:
        Average loss for the epoch
    """
    model.train()
    total_loss = 0

    pbar = tqdm(dataloader, desc="Training")
    for tokens, tools, genres, purposes in pbar:
        tokens = tokens.to(device)
        tools = tools.to(device)
        genres = genres.to(device)
        purposes = purposes.to(device)

        # Forward pass
        _, genre_logits, purpose_logits = model(tokens)

        # For now, skip token-level (tools) - just do doc-level
        # You can add token-level later if needed

        # Compute losses
        genre_loss = criterion(genre_logits, genres)
        purpose_loss = criterion(purpose_logits, purposes)

        total_loss_batch = genre_loss + purpose_loss

        # Backward pass
        optimizer.zero_grad()
        total_loss_batch.backward()

        # Gradient clipping (prevents exploding gradients)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        total_loss += total_loss_batch.item()
        pbar.set_postfix(loss=f"{total_loss_batch.item():.4f}")

    return total_loss / len(dataloader)

@torch.no_grad()
def evaluate(model, dataloader, criterion, device):
    """
    Evaluate model on validation set.

    Returns:
        Average loss
    """
    model.eval()
    total_loss = 0

    for tokens, tools, genres, purposes in dataloader:
        tokens = tokens.to(device)
        genres = genres.to(device)
        purposes = purposes.to(device)

        _, genre_logits, purpose_logits = model(tokens)

        genre_loss = criterion(genre_logits, genres)
        purpose_loss = criterion(purpose_logits, purposes)

        total_loss += (genre_loss + purpose_loss).item()

    return total_loss / len(dataloader)

# ============================================================================
# INFERENCE
# ============================================================================

@torch.no_grad()
def predict(model, readme_text, tokenizer, threshold=0.5):
    """
    Predict labels for a single README.

    Args:
        model: Trained model
        readme_text: Full README content (string)
        tokenizer: Tokenizer
        threshold: Confidence threshold (0.5 = 50%)

    Returns:
        Dictionary with predicted labels and confidence scores
    """
    model.eval()

    # Tokenize
    tokens = tokenizer.encode(readme_text)[:config.BLOCK_SIZE]
    if len(tokens) < config.BLOCK_SIZE:
        tokens += [tokenizer.pad_token_id] * (config.BLOCK_SIZE - len(tokens))

    tokens = torch.tensor([tokens]).to(config.DEVICE)

    # Forward pass
    _, genre_logits, purpose_logits = model(tokens)

    # Convert to probabilities
    genre_probs = torch.sigmoid(genre_logits)[0]
    purpose_probs = torch.sigmoid(purpose_logits)[0]

    # Get predictions above threshold
    predicted_genres = []
    predicted_purposes = []

    for i, prob in enumerate(genre_probs):
        if prob > threshold:
            predicted_genres.append({
                "label": idx2genre[i],
                "confidence": prob.item()
            })

    for i, prob in enumerate(purpose_probs):
        if prob > threshold:
            predicted_purposes.append({
                "label": idx2purpose[i],
                "confidence": prob.item()
            })

    return {
        "genres": predicted_genres,
        "purposes": predicted_purposes
    }

# ============================================================================
# MAIN TRAINING SCRIPT
# ============================================================================

def main():
    """
    Main training function.

    To use:
    1. Create labeled_data.json with your manual labels
    2. Run: python train_portfolio_classifier.py
    3. Model saves to portfolio_classifier.pt
    """

    print("🚀 Portfolio Classifier Training")
    print(f"Device: {config.DEVICE}")
    print(f"Tools: {len(config.TOOL_VOCAB)}")
    print(f"Genres: {len(config.GENRE_VOCAB)}")
    print(f"Purposes: {len(config.PURPOSE_VOCAB)}")
    print()

    # Initialize tokenizer
    tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token

    # Load your NLP model here
    # from your_model_file import NLP, config as model_config
    # model = NLP(model_config,
    #             num_tools=len(config.TOOL_VOCAB),
    #             num_genres=len(config.GENRE_VOCAB),
    #             num_purposes=len(config.PURPOSE_VOCAB))
    # model = model.to(config.DEVICE)

    print("⚠️  Import your NLP model and uncomment the training code!")
    print()
    print("Expected data format:")
    print(json.dumps({
        "name": "Example Project",
        "readme": "Full README text here...",
        "tools": ["django", "react"],
        "genres": ["web-app"],
        "purposes": ["analytics"]
    }, indent=2))

if __name__ == "__main__":
    main()